In [4]:
tmp = !pwd
current_path = tmp[0]

import psycopg2
from psycopg2 import Error
import timeit

conn = ('kevin', '', '127.0.0.1', '5432', 'flightly')

class PostgresQuery(object):
    def __init__(self, query):
        user, password, host, port, database = conn
        self._query = query
        try:
            self._connection = psycopg2.connect(
                user = user,
                password = password,
                host = host,
                port = port,
                database = database
            )
            self._cursor = self._connection.cursor()
        except:
            print("Failed to connect to PostgreSQL DB")

    def query(self, times=1):
        try:
            results = self._run_sql()
            time = timeit.timeit(self._run_sql, number=times)
            self._connection.commit()
            return results, time
        except (Exception, psycopg2.DatabaseError) as error :
            print ("Error while executing SQL command", error)
        finally:
            self._cursor.close()
            self._connection.close()
    
    def _run_sql(self):
        self._cursor.execute(self._query)
        return self._cursor.fetchall()

In [5]:
init_queries = {
    'drop_all_tables': """
        DROP SCHEMA public CASCADE;
        CREATE SCHEMA public;
        """,
    'create_airlines_table': """
        CREATE TABLE airlines (IATA TEXT PRIMARY KEY NOT NULL,
          NAME TEXT);
        """,
    'create_airports_table': """
        CREATE TABLE airports (IATA TEXT PRIMARY KEY NOT NULL,
          NAME TEXT,
          CITY TEXT,
          STATE TEXT,
          COUNTRY TEXT,
          LATITUDE TEXT,
          LONGITUDE TEXT
          );
        """,
    'create_flights_table': """
        CREATE TABLE flights
        (
            year text,
            month text,
            day text,
            day_of_week text,
            airline text,
            flight_number text,
            tail_number text,
            origin_airport text,
            destination_airport text,
            scheduled_departure text,
            departure_time text,
            departure_delay text,
            taxi_out text,
            wheels_off text,
            scheduled_time text,
            elapsed_time text,
            air_time text,
            distance text,
            wheels_on text,
            taxi_in text,
            scheduled_arrival text,
            arrival_time text,
            arrival_delay text,
            diverted text,
            cancelled text,
            cancellation_reason text,
            air_system_delay text,
            security_delay text,
            airline_delay text,
            late_aircraft_delay text,
            weather_delay text
        )
        """,
    'load_airlines_csv': f"""
        COPY airlines
        FROM '{current_path}/csv/airlines.csv' DELIMITER ',' HEADER CSV;
        """,
    'load_airports_csv': f"""
        COPY airports
        FROM '{current_path}/csv/airports.csv' DELIMITER ',' HEADER CSV;
        """,
    'load_flights_csv': f"""
        COPY flights
        FROM '{current_path}/csv/flights_reduced.csv' DELIMITER ',' HEADER CSV;
        """
}

for query in init_queries.items():
    PostgresQuery(query[1])

In [7]:
query, time = PostgresQuery("SELECT tail_number from flights where origin_airport = 'ATL' and airline = 'AA'").query(200)
time

0.719740741999999